In [ ]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////content/dmeyf.db

In [ ]:
# Creo macro para sumar dos valores, si alguno es null, lo considera como 0
%%sql
CREATE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

In [ ]:
# Creo macro para calcular ratio previendo division por cero
%%sql
CREATE MACRO calculo_ratio(a, b) AS NULLIF(a / NULLIF(b,0),0);

In [ ]:
# Creo macro para eliminar el drifting de la variable
%%sql
CREATE MACRO corrijo_drift_rank(a) AS rank() over (partition by foto_mes order by a);

In [ ]:
# Sumo dos variables y creo una nueva
%%sql
select
    Master_msaldototal, 
    Visa_msaldototal, 
    ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01

In [ ]:
# Aplico macro para sumar sin nulls
%%sql
select distinct
    Master_msaldototal, 
    Visa_msaldototal, 
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
from competencia_01

In [ ]:
# Aplico macro para calcular ratios de dos variables
%%sql
select distinct
    Master_msaldototal, 
    Visa_msaldototal, 
    calculo_ratio(Master_msaldototal, Visa_msaldototal) as ratio_master_visa
from competencia_01

In [ ]:
# Eliminar el drifting
%%sql
select
    foto_mes, 
    cliente_antiguedad,
    row_number() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_2,
    percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3,
    cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4,
    ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5,
    ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6,
    ntile(10) over (partition by foto_mes order by cliente_antiguedad desc) as cliente_antiguedad_7
from competencia_01
order by cliente_antiguedad

In [ ]:
# Aplico macro para eliminar el drifting
%%sql
select
    foto_mes, 
    cliente_antiguedad,
    corrijo_drift_rank(Master_msaldototal) as Master_msaldototal_sin_drift
from competencia_01
order by cliente_antiguedad

In [ ]:
# Feature que muestre el valor del periodo anterior
%%sql
select
  numero_de_cliente, 
  foto_mes, 
  ctrx_quarter, 
  lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_02
limit 10

In [ ]:
# Media móvil en los últimos 6 meses
%%sql
select
  numero_de_cliente, 
  foto_mes, 
  ctrx_quarter, 
  lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter, 
  ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter, 
  ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_02
limit 10

In [ ]:
# Simplificación del código previo para trabajar en la ventana definida y no tener que repetir para cada linea.
%%sql
select
  numero_de_cliente, 
  foto_mes, 
  ctrx_quarter, 
  avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6, 
  max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6, 
  min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

In [ ]:
# Para calcular la pendiente. El beta de la relación entre variables
%%sql
select
  numero_de_cliente, 
  foto_mes, 
  ctrx_quarter,
  regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

Para realizar estas operaciones en un conjunto de variables... puedo usar una lista de variables y aplicarles las funciones.

In [ ]:
campos = [
    'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 
    'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 
    'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 
    'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 
    'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 
    'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 
    'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 
    'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 
    'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 
    'cseguro_accidentes_personales', 'ccaja_seguridad', 'cpayroll_trx', 'mpayroll', 'mpayroll2', 'cpayroll2_trx', 
    'ccuenta_debitos_automaticos', 'mcuenta_debitos_automaticos', 'ctarjeta_visa_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 
    'ctarjeta_master_debitos_automaticos', 'mttarjeta_master_debitos_automaticos', 'cpagodeservicios', 'mpagodeservicios', 'cpagomiscuentas', 
    'mpagomiscuentas', 'ccajeros_propios_descuentos', 'mcajeros_propios_descuentos', 'ctarjeta_visa_descuentos', 'mtarjeta_visa_descuentos', 
    'ctarjeta_master_descuentos', 'mtarjeta_master_descuentos', 'ccomisiones_mantenimiento', 'mcomisiones_mantenimiento', 'ccomisiones_otras', 
    'mcomisiones_otras', 'cforex', 'cforex_buy', 'mforex_buy', 'cforex_sell', 'mforex_sell', 'ctransferencias_recibidas', 'mtransferencias_recibidas', 
    'ctransferencias_emitidas', 'mtransferencias_emitidas', 'cextraccion_autoservicio', 'mextraccion_autoservicio', 'ccheques_depositados', 
    'mcheques_depositados', 'ccheques_emitidos', 'mcheques_emitidos', 'ccheques_depositados_rechazados', 'mcheques_depositados_rechazados', 
    'ccheques_emitidos_rechazados', 'mcheques_emitidos_rechazados', 'tcallcenter', 'ccallcenter_transacciones', 'thomebanking', 
    'chomebanking_transacciones', 'ccajas_transacciones', 'ccajas_consultas', 'ccajas_depositos', 'ccajas_extracciones', 'ccajas_otras', 
    'catm_trx', 'matm', 'catm_trx_other', 'matm_other', 'ctrx_quarter', 'Master_delinquency', 'Master_status', 'Master_mfinanciacion_limite', 
    'Master_Fvencimiento', 'Master_Finiciomora', 'Master_msaldototal', 'Master_msaldopesos', 'Master_msaldodolares', 'Master_mconsumospesos', 
    'Master_mconsumosdolares', 'Master_mlimitecompra', 'Master_madelantopesos', 'Master_madelantodolares', 'Master_fultimo_cierre', 'Master_mpagado', 
    'Master_mpagospesos', 'Master_mpagosdolares', 'Master_fechaalta', 'Master_mconsumototal', 'Master_cconsumos', 'Master_cadelantosefectivo', 
    'Master_mpagominimo', 'Visa_delinquency', 'Visa_status', 'Visa_mfinanciacion_limite', 'Visa_Fvencimiento', 'Visa_Finiciomora', 'Visa_msaldototal', 
    'Visa_msaldopesos', 'Visa_msaldodolares', 'Visa_mconsumospesos', 'Visa_mconsumosdolares', 'Visa_mlimitecompra', 'Visa_madelantopesos', 
    'Visa_madelantodolares', 'Visa_fultimo_cierre', 'Visa_mpagado', 'Visa_mpagospesos', 'Visa_mpagosdolares', 'Visa_fechaalta', 'Visa_mconsumototal', 
    'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo']

In [ ]:
# Para realizar estas operaciones en un conjunto de variables... puedo usar una lista de variables y aplicarles las funciones.
%%sql
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_6 as ctrx_{campo}_slope_6"
print(nuevos_features)

#### Aplicando funciones para Competencia02

In [ ]:
%%sql
CREATE TABLE table_1 AS
SELECT
    *,
    
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as tc_transacciones_total,
    suma_sin_null(ctarjeta_visa, ctarjeta_master) as tc_ctarjeta_total,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as tc_consumo_total,
    suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as tc_mconsumo_total,
    suma_sin_null(Visa_mlimitecompra, Master_mlimitecompra) as tc_limcompra_total,
    suma_sin_null(Master_mpagado, Visa_mpagado) as tc_pagado_total,
    suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as tc_saldo_total,
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_pagomin_total,
    ifnull(mprestamos_personales, 0) + ifnull(mprestamos_prendarios, 0) + ifnull(mprestamos_hipotecarios, 0) as pasivos_total,
    ifnull(mplazo_fijo_dolares, 0) + ifnull(mplazo_fijo_pesos, 0) + ifnull(minversion1_pesos, 0) 
        + ifnull(minversion1_dolares, 0) + ifnull(minversion2, 0) as activos_total,
    ifnull(mpayroll, 0) + ifnull(mpayroll2, 0) as flujos_ingresos_total,
    ifnull(mcajeros_propios_descuentos, 0) + ifnull(mtarjeta_visa_descuentos, 0) + ifnull(mtarjeta_master_descuentos, 0) as beneficios_total,
    ifnull(mcomisiones_mantenimiento, 0) + ifnull(mcomisiones_otras, 0) as comisiones_total,
    ifnull(mcuenta_debitos_automaticos, 0) + ifnull(mttarjeta_visa_debitos_automaticos, 0) + ifnull(mttarjeta_master_debitos_automaticos, 0) 
        + ifnull(mpagodeservicios, 0) + ifnull(mpagomiscuentas, 0) as gastos_total,


    corrijo_drift_rank(tc_mconsumo_total) as tc_mconsumo_total_sin_drift,
    corrijo_drift_rank(tc_limcompra_total) as tc_limcompra_total_sin_drift,
    corrijo_drift_rank(tc_pagado_total) as tc_pagado_total_sin_drift,
    corrijo_drift_rank(pasivos_total) as pasivos_total_sin_drift,
    corrijo_drift_rank(activos_total) as activos_total_sin_drift,
    corrijo_drift_rank(flujos_ingresos_total) as flujos_ingresos_total_sin_drift,
    corrijo_drift_rank(comisiones_total) as comisiones_total_sin_drift,
    corrijo_drift_rank(gastos_total) as gastos_total_sin_drift,
    

    lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as ctrx_quarter_sin_drift,
    lag(tc_mconsumo_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tc_mconsumo_total_sin_drift,
    lag(tc_limcompra_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tc_limcompra_total_sin_drift,
    lag(tc_pagado_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tc_pagado_total_sin_drift,
    lag(pasivos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_pasivos_total_sin_drift,
    lag(activos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_activos_total_sin_drift,
    lag(flujos_ingresos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_flujos_ingresos_total_sin_drift,
    lag(comisiones_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_comisiones_total_sin_drift,
    lag(gastos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_gastos_total_sin_drift,


    calculo_ratio(tc_consumo_total, flujos_ingresos_total) as consumo_sobre_ingresos,
    calculo_ratio(tc_mconsumo_total, flujos_ingresos_total) as tcm_consumo_sobre_ingresos,
    calculo_ratio(tc_pagado_total, flujos_ingresos_total) as pagado_sobre_ingresos,
    calculo_ratio(tc_pagado_total, activos_total) as pagado_sobre_activos,
    calculo_ratio(tc_pagomin_total, flujos_ingresos_total) as pagomin_sobre_ingresos,
    calculo_ratio(tc_consumo_total, activos_total) as gastos_sobre_activos,
    calculo_ratio(activos_total, pasivos_total) as activos_sobre_pasivos,
    calculo_ratio(beneficios_total, comisiones_total) as beneficios_sobre_comisiones,
    calculo_ratio(beneficios_total, tc_consumo_total) as beneficios_sobre_consumos,
    calculo_ratio(gastos_total, flujos_ingresos_total) as gastos_sobre_ingresos,
    calculo_ratio(tc_limcompra_total, tc_consumo_total) as lim_sobre_consumos,
    
    
    avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6, 
    max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6, 
    min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6
    regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
    
    avg(cproductos) over ventana_6 as cproductos_media_6, 
    max(cproductos) over ventana_6 as cproductos_max_6, 
    min(cproductos) over ventana_6 as cproductos_min_6
    regr_slope(cproductos, cliente_antiguedad) over ventana_6 as cproductos_slope_6
    
    avg(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_transacciones_media_6, 
    max(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_transacciones_max_6, 
    min(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_transacciones_min_6
    regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_debito_transacciones_slope_6

    avg(chomebanking_transacciones) over ventana_6 as chomebanking_transacciones_media_6, 
    max(chomebanking_transacciones) over ventana_6 as chomebanking_transacciones_max_6, 
    min(chomebanking_transacciones) over ventana_6 as chomebanking_transacciones_min_6
    regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_6 as chomebanking_transacciones_slope_6

    avg(tc_transacciones_total) over ventana_6 as tc_transacciones_total_media_6, 
    max(tc_transacciones_total) over ventana_6 as tc_transacciones_total_max_6, 
    min(tc_transacciones_total) over ventana_6 as tc_transacciones_total_min_6
    regr_slope(tc_transacciones_total, cliente_antiguedad) over ventana_6 as tc_transacciones_total_slope_6    
     
FROM competencia_01
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)


In [1]:
%%sql
ALTER TABLE table_1 DROP COLUMN ctarjeta_visa_transacciones;
ALTER TABLE table_1 DROP COLUMN ctarjeta_master_transacciones;
ALTER TABLE table_1 DROP COLUMN ctarjeta_visa;
ALTER TABLE table_1 DROP COLUMN ctarjeta_master;
ALTER TABLE table_1 DROP COLUMN mtarjeta_visa_consumo;
ALTER TABLE table_1 DROP COLUMN mtarjeta_master_consumo;
ALTER TABLE table_1 DROP COLUMN Master_mconsumototal;
ALTER TABLE table_1 DROP COLUMN Visa_mconsumototal;
ALTER TABLE table_1 DROP COLUMN Visa_mlimitecompra;
ALTER TABLE table_1 DROP COLUMN Master_mlimitecompra;
ALTER TABLE table_1 DROP COLUMN Master_mpagado;
ALTER TABLE table_1 DROP COLUMN Visa_mpagado;
ALTER TABLE table_1 DROP COLUMN Master_mpagominimo;
ALTER TABLE table_1 DROP COLUMN Visa_mpagominimo;
ALTER TABLE table_1 DROP COLUMN tc_mconsumo_total;
ALTER TABLE table_1 DROP COLUMN tc_limcompra_total;
ALTER TABLE table_1 DROP COLUMN tc_pagado_total;
ALTER TABLE table_1 DROP COLUMN pasivos_total;
ALTER TABLE table_1 DROP COLUMN activos_total;
ALTER TABLE table_1 DROP COLUMN flujos_ingresos_total;
ALTER TABLE table_1 DROP COLUMN comisiones_total;
ALTER TABLE table_1 DROP COLUMN gastos_total;


UsageError: Cell magic `%%sql` not found.


In [ ]:
%%sql
copy table_1 to '/content/competencia_02.csv.gz' (FORMAT CSV, HEADER)